# Most Dangerous Streets

In [1]:
import pytz
import folium
import pandas as pd
from datetime import datetime, timedelta
from crash_data_analysis import CrashDataAnalysis

In [2]:
cda = CrashDataAnalysis()

In [3]:
start_date = pytz.timezone('America/Denver').localize(datetime(2013,1,1))
# end_date = pytz.timezone('America/Denver').localize(datetime(2024,2,1))
# most_recent_crash = cda.most_recent_crash_timestamp()
end_date = cda.most_recent_crash_timestamp()

street_crashes = cda.street_metrics(start_date, end_date)
street_crashes

,gid,lrsroute,fullname,length_miles,street_line,num_crashes,num_sbi,num_fatality,num_sbi_or_fatality,days_in_data,days_between_crashes,crashes_per_mile_per_week
2133,3900,SPEERN BD,N SPEER BLVD,2.634723,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",3155,67,6,73,3726,1.180983,2.249674
1150,1714,LINCOLN ST4,N LINCOLN ST,2.165535,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",1889,35,2,37,3726,1.972472,1.638785
590,880,BROADWAY3,N BROADWAY,3.331934,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",2842,68,15,83,3726,1.311049,1.602444
1319,1957,MARKET ST,MARKET ST,0.920197,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",766,14,0,14,3726,4.864230,1.563879
4215,7584,SFEDERAL BD1,S FEDERAL BLVD,4.373528,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",3622,92,18,110,3726,1.028713,1.555864
...,...,...,...,...,...,...,...,...,...,...,...,...
474,680,88TH AV3,E 88TH AVE,3.208894,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",1,0,0,0,3726,3726.000000,0.000585
1086,1623,HARDWRICKST-I,N HARDWRICK ST,3.557250,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",1,0,0,0,3726,3726.000000,0.000528
538,788,64TH AV3,E 64TH AVE,3.996461,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",1,0,0,0,3726,3726.000000,0.000470
1401,2114,PENAWB BD,PENA BLVD OUTBOUND,10.850493,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",2,0,0,0,3726,1863.000000,0.000346


In [4]:
days_in_data = (end_date - start_date).days

In [5]:
street_crashes = street_crashes[street_crashes.length_miles > 0.5].copy()

In [6]:
street_crashes['days_between_crashes'] = days_in_data / street_crashes['num_crashes']

In [7]:
street_crashes['crashes_per_mile_per_week'] = (
    (street_crashes['num_crashes'] / street_crashes['length_miles']) / (days_in_data/7))

In [8]:
street_crashes[['gid', 'fullname', 'length_miles', 'num_crashes', 'crashes_per_mile_per_week']].sort_values(
    by='crashes_per_mile_per_week', ascending=False).head(20)

,gid,fullname,length_miles,num_crashes,crashes_per_mile_per_week
2133,3900,N SPEER BLVD,2.634723,3155,2.249674
1150,1714,N LINCOLN ST,2.165535,1889,1.638785
590,880,N BROADWAY,3.331934,2842,1.602444
1319,1957,MARKET ST,0.920197,766,1.563879
4215,7584,S FEDERAL BLVD,4.373528,3622,1.555864
2363,4225,20TH ST,1.453330,1192,1.540875
2367,4229,22ND ST,1.079568,885,1.540099
3139,5384,W COLFAX AVE,2.317662,1805,1.463128
684,1012,E COLFAX AVE,5.475202,4216,1.446624
3794,6796,W ALAMEDA AVE,3.508448,2599,1.391703


In [9]:
this_map = folium.Map(prefer_canvas=True, tiles='Stamen Toner')

for bad_street in street_crashes.sort_values(by='crashes_per_mile_per_week', ascending=False).head(20)['street_line']:
    folium.GeoJson(bad_street, style_function=lambda x: {'color': 'red'}).add_to(this_map)
    
this_map.fit_bounds(this_map.get_bounds())
this_map

## High Injury Network

In [10]:
street_crashes = street_crashes.sort_values(by='num_sbi_or_fatality', ascending=False)

street_crashes['high_injury_perc'] = (
    street_crashes['num_sbi_or_fatality'] / street_crashes['num_sbi_or_fatality'].sum()
)

street_crashes['high_injury_running_sum'] = street_crashes['high_injury_perc'].cumsum()

street_crashes['high_injury_network'] = street_crashes['high_injury_running_sum'] < 0.5

In [11]:
street_crashes.loc[street_crashes.high_injury_network
    , ['gid', 'fullname', 'length_miles', 'num_sbi_or_fatality', 'high_injury_running_sum']]

,gid,fullname,length_miles,num_sbi_or_fatality,high_injury_running_sum
684,1012,E COLFAX AVE,5.475202,128,0.026976
2710,4689,N FEDERAL BLVD,5.367276,112,0.050580
4215,7584,S FEDERAL BLVD,4.373528,110,0.073762
688,1016,N COLORADO BLVD,5.184503,90,0.092729
590,880,N BROADWAY,3.331934,83,0.110221
...,...,...,...,...,...
72,104,E 11TH AVE,0.992406,15,0.486617
1168,1742,N LOGAN ST,2.147136,15,0.489779
86,126,E 16TH AVE,1.627535,15,0.492940
194,259,E 46TH SOUTH AVE,1.376707,15,0.496101


In [12]:
this_map = folium.Map(prefer_canvas=True, tiles='Stamen Toner')

for bad_street in street_crashes.loc[street_crashes.high_injury_network, 'street_line']:
    folium.GeoJson(bad_street, style_function=lambda x: {'color': 'red'}).add_to(this_map)
    
this_map.fit_bounds(this_map.get_bounds())
this_map

In [13]:
street_crashes[street_crashes['fullname'].str.contains('PENA')]

,gid,lrsroute,fullname,length_miles,street_line,num_crashes,num_sbi,num_fatality,num_sbi_or_fatality,days_in_data,days_between_crashes,crashes_per_mile_per_week,high_injury_perc,high_injury_running_sum,high_injury_network
1401,2114,PENAWB BD,PENA BLVD OUTBOUND,10.850493,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",2,0,0,0,3726,1863.0,0.000346,0.0,1.0,False
4014,7264,R105,NB E470 TO EB PENA RAMP,0.636255,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",1,0,0,0,3726,3726.0,0.002953,0.0,1.0,False
1399,2104,PENA BD2,PENA BLVD INBOUND,13.396102,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",2,0,0,0,3726,1863.0,0.000280,0.0,1.0,False


## High Death Network

In [15]:
street_crashes = street_crashes.sort_values(by='num_fatality', ascending=False)

street_crashes['fatality_perc'] = (
    street_crashes['num_fatality'] / street_crashes['num_fatality'].sum()
)

street_crashes['fatality_perc_running_sum'] = street_crashes['fatality_perc'].cumsum()

street_crashes['fatality_network'] = street_crashes['fatality_perc_running_sum'] < 0.5

In [17]:
street_crashes.loc[street_crashes.fatality_network
    , ['gid', 'fullname', 'length_miles', 'num_fatality', 'fatality_perc_running_sum']]

,gid,fullname,length_miles,num_fatality,fatality_perc_running_sum
688,1016,N COLORADO BLVD,5.184503,21,0.035714
4215,7584,S FEDERAL BLVD,4.373528,18,0.066327
2710,4689,N FEDERAL BLVD,5.367276,15,0.091837
590,880,N BROADWAY,3.331934,15,0.117347
3134,5379,W COLFAX AVE,3.567320,13,0.139456
3139,5384,W COLFAX AVE,2.317662,12,0.159864
3794,6796,W ALAMEDA AVE,3.508448,11,0.178571
2123,3890,N SHERIDAN BLVD,5.135898,9,0.193878
3570,6481,S SHERIDAN BLVD,5.380667,8,0.207483
755,1122,N CHAMBERS RD,1.758017,7,0.219388


In [18]:
street_crashes.fatality_network.sum()

50

In [19]:
street_crashes.num_fatality.sum()

588

In [20]:
this_map = folium.Map(prefer_canvas=True, tiles='Stamen Toner')

for bad_street in street_crashes.loc[street_crashes.fatality_network, 'street_line']:
    folium.GeoJson(bad_street, style_function=lambda x: {'color': 'red'}).add_to(this_map)
    
this_map.fit_bounds(this_map.get_bounds())
this_map